In [1]:
import pandas as pd
import numpy as np

import pickle

from tqdm import tqdm

from refined.processor import Refined

In [9]:
refined = Refined.from_pretrained(model_name='wikipedia_model', 
                            entity_set="wikipedia",
                            data_dir="/home/yirenl2/PLM_DC/data-preparator-for-EM/data/refined/wikipedia", 
                            download_files=True,
                            use_precomputed_descriptions=True,
                            device="cuda:0")


model {'s3_bucket': 'refined.public', 's3_key': '2022_aug/wikipedia_model/model.pt', 'local_filename': 'wikipedia_model/model.pt'}
model_config {'s3_bucket': 'refined.public', 's3_key': '2022_aug/wikipedia_model/config.json', 'local_filename': 'wikipedia_model/config.json'}
model_description_embeddings_wikidata {'s3_bucket': 'refined.public', 's3_key': '2022_aug/wikipedia_model/precomputed_entity_descriptions_emb_wikidata_33831487-300.np', 'local_filename': 'wikipedia_model/precomputed_entity_descriptions_emb_wikidata_33831487-300.np'}
model_description_embeddings_wikipedia {'s3_bucket': 'refined.public', 's3_key': '2022_aug/wikipedia_model/precomputed_entity_descriptions_emb_wikipedia_6269457-300.np', 'local_filename': 'wikipedia_model/precomputed_entity_descriptions_emb_wikipedia_6269457-300.np'}
wiki_pem {'s3_bucket': 'refined.public', 's3_key': '2022_aug/wikipedia_data/wiki_pem.json', 'local_filename': 'wikipedia_data/wiki_pem.json'}
class_to_label {'s3_bucket': 'refined.public', '

Loading PEM:   0%|          | 0/18000000.0 [00:00<?, ?it/s]

22:00:57 - utilities.lookup_utils - INFO - Loading enwiki sitelinks


Loading enwiki sitelinks:   0%|          | 0/8000000.0 [00:00<?, ?it/s]

22:01:25 - utilities.lookup_utils - INFO - Loaded enwiki_to_qcode, size = 8532272
temp
22:01:27 - utilities.lookup_utils - INFO - Loading subclasses


Loading Wikidata subclasses:   0%|          | 0/2000000.0 [00:00<?, ?it/s]

22:06:08 - utilities.lookup_utils - INFO - Loaded subclasses, size = 2489258
22:06:08 - utilities.lookup_utils - INFO - Loading qcode_to_idx


Loading qcode_to_idx:   0%|          | 0/6000000 [00:00<?, ?it/s]

22:06:26 - doc_preprocessing.doc_preprocessor - INFO - Loaded all
Reading description embeddings memory map /home/yirenl2/PLM_DC/data-preparator-for-EM/data/refined/wikipedia/wikipedia_model/precomputed_entity_descriptions_emb_wikipedia_6269457-300.np


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.we

In [32]:
# text = "ASICS Tiger GEL-Kayano Trainer Evo Origami Pack - Black / Dark Grey"
text = "Rolex GMT Master II 116710 LN"
spans = refined.process_text(text)

for i in range(len(spans)-1, -1, -1):
    # wrap the string in the span with the span type
    span = spans[i]
    spanType = span.pred_types[0][1]
    text = text[:span.start] + '<' + spanType + '>' + text[span.start:span.start + span.ln] + '</' + spanType + '>' + text[span.start + span.ln:]
    print(
        '\n'.join([
            span.text,
            str((span.start, span.start + span.ln)),
            span.coarse_type,
            str(span.candidate_entity_ids),
        ])
    )
#     print(span)
print(text)

Master II
(10, 19)
MENTION
[('Q3314082', 1.0)]
Rolex GMT
(0, 9)
MENTION
[('Q382707', 1.0)]
<artificial physical object>Rolex GMT</artificial physical object> <type>Master II</type> 116710 LN


In [2]:
split = 'train'

In [3]:
dSpan= pickle.load(open('./%s_refined_outputs.pkl'%split, 'rb'))
df = pd.read_csv("%s.txt"%split, sep='\t', header=None)
df.columns = ['r1', 'r2', 'isDup']


In [4]:
df.head(3)

,r1,r2,isDup
0,COL name VAL lg 24 ' lds4821ww semi integrated...,COL name VAL lg ldf6920bb fully integrated dis...,0
1,COL name VAL speck seethru clear hard shell ca...,COL name VAL speck products seethru case for a...,0
2,COL name VAL denon blu-ray disc dvd/cd player ...,COL name VAL denon dvd-2930ci dvd player dvd29...,0


In [5]:
df.iloc[0]['r1']

"COL name VAL lg 24 ' lds4821ww semi integrated built in white dishwasher lds4821wh COL description VAL lg 24 ' lds4821ww semi integrated built in white dishwasher lds4821wh xl tall tub cleans up to 16 place settings at once adjustable upper rack lodecibel quiet operation senseclean wash system 4 wash cycles with 3 spray arms multi-level water direction slim direct drive motor semi-integrated electronic control panel white finish COL price VAL  "

In [6]:
lineNum = 5
recId = 'r1'
spanNum = 0

span = dSpan[lineNum][recId][spanNum]
print(
    '\n'.join([
        span.text,
        str((span.start, span.start + span.ln)),
        span.coarse_type,
        str(span.pred_types[0]),
    ])
)


VAL onkyo
(9, 18)
MENTION
('Q4830453', 'business', 0.7604)


In [7]:
text = df.iloc[lineNum][recId]

for i in range(len(dSpan[lineNum][recId])-1, -1, -1):
    # wrap the string in the span with the span type
    span = dSpan[lineNum][recId][i]
    spanType = span.pred_types[0][1]
    text = text[:span.start] + '<' + spanType + '>' + text[span.start:span.start + span.ln] + '</' + spanType + '>' + text[span.start + span.ln:]

In [8]:
text

'COL name <business>VAL onkyo</business> thx select2 plus certified 7.1 channel home theater black receiver txsr706b COL description <business>VAL onkyo</business> thx select2 plus certified 7.1 channel home theater black receiver txsr706b 100 watts per channel all discrete circuitry linear optimum gain volume <business>dolby</business> digital plus , <business>dolby</business> truehd and dts-hd decoding hdmi version v1 .3 a repeater dsd processing <business>audyssey</business> dynamic eq accurate speaker distance setting hdmi deep color capable ( 36bit ) analog to hdmi 1080p upconversion <business>faroudja</business> dcd<class>i</class> cinema black finish COL price VAL  '